# Prediction of music listening (Part II): Recommendation with Alternating Least Squares

In this notebook, we aim to make predictions based on the Weighted Alternating Least Squares model. The outputs of the WALS can be used as a latent representation for both users and items. 

We feed these latent representation to various ML models in notebook III.


You should note that this notebook and all the following were implemented on Google Colab, which is quicker than Jupyter in our case. If you want to open a Google Colab session, [here is the link](https://colab.research.google.com/notebooks/welcome.ipynb#recent=true).


### Package installation


*   **Implicit** is a package used to provide fast Python implementations of several different popular recommendation algorithms for implicit feedback datasets. It enables us to implement Alternating Least Squares.
*   **Kaggle** is used to import data from Kaggle (our dataset is from a Kaggle challenge)
*   **Pandas v.0.21** is a package providing fast, flexible, and expressive data structures designed to make working with “relational” or “labeled” data both easy and intuitive.

To import a Kaggle dataset, you have to generate a token. In order to do so,you need to have a Kaggle account for accessing Kaggle API. You can find the token on your Kaggle account page. Once you download the token to your local machine, you can copy the token in the notebook. [Here is the link to access the Kaggle page of the challenge.](https://www.kaggle.com/c/kkbox-music-recommendation-challenge/overview)




In [0]:
!pip install --user implicit
!pip install --user -q kaggle
!pip install --user pandas==0.21 ## This version is necessary for implicit package to work

import warnings
warnings.filterwarnings('ignore')

After that, we want to load the dataset, create a directory and copy the data from Kaggle. In the provided dataset, we will use train.csv (and unzip it).

The test set is mainly for competition submissions and thus doesn't contain targets. 

Therefore we can't use the test set for local model evaluation.

We split the train set in order to do our local evaluation.



In [0]:
## You need to download a kaggle token from your personal computer in order to download dataset from kaggle
## HELP ==>
#https://adityashrm21.github.io/Setting-Up-Kaggle/

from google.colab import files
files.upload()

In [0]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!kaggle competitions download -c kkbox-music-recommendation-challenge
!7za x train.csv.7z # unzip the file

## Rearanging data
In order to be able to evaluate our results, we must compare datasets which are "similar". In the test dataset, we only want to have songs that appear in the training dataset. If not, we couldn't predict the target.

In [0]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [0]:
df = pd.read_csv('train.csv') # read the data
df_train, df_test = train_test_split(df, test_size = 0.2) #split into training and test dataset

In [0]:
#Clear out tuples with either song or user that didn't figure in the training dataset.

df_test = df_test[df_test['song_id'].isin(df_train['song_id'])]
df_test = df_test[df_test['msno'].isin(df_train['msno'])]

print('df_test shape before cleaning: ', df_test.shape[0])
print('df_test shape after cleaning: ', df_test.shape[0])

In [0]:
# How many null values do we have in each column ?

percent_missing = df_test.isnull().sum() * 100 / len(df_test)
missing_value_df = pd.DataFrame({'column_name': df_test.columns,
                                 'percent_missing': percent_missing})
print ('Number of missing values per column of the test set')
missing_value_df

Another problem we are facing is the usability of the data provided. Initialy, UserID are strings, as SongID. We rename the colums and transform it into numerical attributes, that are easier to manipulate.


In a second time, we need to have the same indexes in both the test dataset and the training dataset, to be able to compare the two.

In [0]:
df_train['SongID'] = df_train.groupby(['song_id']).ngroup()
df_train['UserID'] = df_train.groupby(['msno']).ngroup()

print('We have ', df_train['UserID'].nunique(), 'unique users.')
print('We have ', df_train['SongID'].nunique(), 'unique songs.')

**Note:** One problem we stumbled upon when using the IMPLICIT package is that User_ids and song ids must be continuous in values. This is why we have to get set IDs for training first then merge the Test set with training on those ids.


In [0]:
# The indexes must be the same in training and testing datasets

df_train_ids = df_train.groupby('song_id').first().reset_index()
df_train_ids = df_train_ids[['msno', 'song_id', 'SongID', 'UserID']]

df_test = df_test[['target', 'song_id', 'msno']]

df_test_ids = pd.merge(df_test, df_train_ids, left_on = 'msno', right_on = 'msno').drop(['msno'], axis=1)
df_test_ids = pd.merge(df_test, df_train_ids, left_on = 'song_id', right_on = 'song_id').drop(['song_id'], axis=1)

df_train = df_train[['UserID', 'SongID', 'target']]

df_test = df_test_ids[['UserID', 'SongID', 'target']]

print('df test shape = ',df_test.shape)

## Alternating Least Squares & latent factors
In order to use Alternating Least Squares more efficiently, we will transform the training dataset into a CSR matrix. 


CSR stands for Compressed Sparse Row matrix. Sparse matrices can be used in efficient arithmetic operations: they support addition, subtraction, multiplication, division, and matrix power.

In this section, we will use latent-factor models. They try to explain observed interactions between large numbers of users and songs through a relatively small number of observations.

First, we formulate the learning problem as a matrix completion problem. Then, we will use a type of matrix factorization model to "fill in" the blanks. We are given implicit ratings that users have given certain items (if they listened a song again or not) and our goal is to predict their ratings for the rest of the items. Formally, if there are $n$ users and $m$ items, we are given an $n \times m$ matrix $R$ in which the generic entry $(u, i)$ represents the rating for item $i$ by user $u$. Matrix $R$ has many missing entries indicating unobserved ratings, and our task is to estimate these unobserved ratings.

A popular approach to the matrix completion problem is matrix factorization, where we want to "summarize" users and items with their latent factors. For that, we approximate the initial matrix $R$ by the product of two smaller matrices $X$ and $Y$.

The challenge is to calculate $X$ and $Y$. We do this iteratively: knowing $Y$, we can calculate the best value of $X$, and vice versa. It means from the initial values of $X$ and $Y$ in the beginning, we calculate the best $X$ according to $Y$, and then calculate the best $Y$ according to the new $X$. This process is repeated until the distance from $XY$ to $R$ is small.

The values composing $X$ and $Y$ are called latent factors.

To use Alternating Least Squares, we first need to intialize the model, then train it on the sparse matrix we just created.

In [0]:
from scipy.sparse import csr_matrix
import implicit
import multiprocessing

We are using the library **multiprocessing**.  This package offers both local and remote concurrency, solving Global Interpreter problems by using subprocesses instead of threads. Due to this, the multiprocessing module allows the programmer to fully leverage multiple processors on a given machine.

In [0]:
user_item_data = csr_matrix((df_train['target'], (df_train['UserID'], df_train['SongID'])), \
                           shape = (df_train['UserID'].nunique(), df_train['SongID'].nunique()))

user_item_data.shape

In [0]:
# Initialize 
model = implicit.als.AlternatingLeastSquares(factors = 64)

# Train
model.fit(user_item_data)

In [0]:
#model.recommend_all(user_item_data,10)  This function would have saved us a lot of trouble but it doesnt work

Now we want to do user re-listenings predictions. 

To do so, we check if the song figures in the N top recommended songs for the user, and if so, we suppose it's target is 1.

Unfortunately this approach was inconclusive as our model will always predict false even for N= 1000000


In [85]:
## This function return a boolean to know if the current song is recommended for the current user

## It takes in input a song, a user, and the number of recommendations to genrate


def predict(user_id, song_id,filter_already_liked_items=False, N = 10000):
    r = model.recommend(user_id, user_item_data, N = N)
    recommendations = [i[0] for i in r] #  get recommended songID without the score associated
    return (song_id in recommendations)


predict (5951,260925) ## This is a sample from the test dataset (it should be True)


False

In [86]:

args=[[UserID,SongID] for UserID, SongID, target in df_test.sample(frac=0.01).head(1000).values]

#multiprocessing
try:
    cpus = multiprocessing.cpu_count()
except NotImplementedError:
    cpus = 2   # default
print('cpus = ', cpus)

pool = multiprocessing.Pool(processes = cpus) # start multiple worker processes

predictions=pool.starmap(predict, args)

print(predictions)

cpus =  4
[False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, Fa

In this part we get the latent item / user representations and merge them with the dataset. The resulting dataframe is used in next notebooks to make predictions.

In [87]:
item_latent = pd.DataFrame(model.item_factors)
user_latent = pd.DataFrame(model.user_factors)
user_latent.columns = ['user_latent' + str(x) for i, x in enumerate(user_latent.columns, 1)]
item_latent.columns = ['item_latent' + str(x) for i, x in enumerate(item_latent.columns, 1)]
item_latent['ID'] = item_latent.index
user_latent['ID'] = user_latent.index

user_latent.head()

,user_latent0,user_latent1,user_latent2,user_latent3,user_latent4,user_latent5,user_latent6,user_latent7,user_latent8,user_latent9,...,user_latent55,user_latent56,user_latent57,user_latent58,user_latent59,user_latent60,user_latent61,user_latent62,user_latent63,ID
0,-2.644544e-12,-9.249792e-13,-3.224912e-13,-9.439513e-13,7.236304e-13,-8.306632e-13,9.881727e-13,-8.107907e-13,-1.547658e-12,-2.536470e-12,...,3.000360e-12,8.705412e-13,-1.765695e-12,2.107446e-13,1.371772e-12,1.115754e-12,2.426915e-13,1.950971e-12,-4.237625e-13,0
1,-6.598627e-13,-1.196054e-12,-1.430170e-12,-4.840771e-13,6.661880e-13,4.118457e-13,4.349186e-13,-1.709455e-12,-1.480153e-12,-1.868602e-12,...,1.899065e-12,3.959238e-13,3.158488e-13,1.452306e-12,1.577157e-12,-2.245885e-14,2.923200e-13,2.977683e-12,8.951582e-13,1
2,8.087132e-11,3.323642e-11,1.069165e-11,3.920881e-11,2.010811e-11,1.108030e-11,4.999447e-11,4.102699e-12,6.302878e-11,1.389113e-10,...,-2.744835e-11,4.669810e-11,-4.482312e-11,-1.043379e-11,-1.895363e-10,-7.028418e-11,2.652573e-12,-3.908648e-11,2.607690e-11,2
3,-1.644166e-12,3.255939e-11,3.780337e-12,1.902603e-12,-1.780760e-11,-1.469769e-11,6.033555e-12,4.034762e-11,6.017527e-11,2.160263e-11,...,-2.878170e-11,4.839207e-12,1.866528e-11,-1.427836e-11,-3.688095e-11,-2.994064e-11,-1.850649e-11,-6.332652e-11,-7.296884e-12,3
4,-4.988765e-12,-5.361404e-12,-5.259260e-12,-3.595482e-12,6.883490e-12,-3.511613e-13,6.941035e-12,-6.516514e-12,-9.911166e-12,-6.946741e-12,...,1.433093e-11,5.181223e-12,-3.812514e-13,5.413257e-12,5.837914e-12,1.335183e-12,2.625753e-12,9.709965e-12,4.123598e-12,4


In [88]:
df_mini = df_test.sample(frac = 1)
print('df mini shape = ', df_mini.shape)
print('df mini size = ', df_mini.count())

df_latent = pd.merge(df_mini, item_latent, left_on = 'UserID', right_on = 'ID').drop(['ID'], axis = 1)
df_latent = pd.merge(df_latent, user_latent, left_on = 'SongID', right_on = 'ID').drop(['ID'], axis = 1)
print('df latent shape after second merge: ', df_latent.shape)

df mini shape =  (1436893, 3)
df mini size =  UserID    1436893
SongID    1436893
target    1436893
dtype: int64
df latent shape after second merge:  (1436893, 131)
